In [1]:
import os
import yaml
from pathlib import Path
import shutil

In [2]:
import torch
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\junji\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
if torch.cuda.is_available():
    print("PyTorch使用的是GPU版本")
else:
    print("PyTorch使用的是CPU版本")

PyTorch使用的是GPU版本


## 加载数据

In [5]:
data_dir=Path("data")
"""读取类别信息"""
classes_file = data_dir / "classes.txt"
with open(classes_file, 'r', encoding='utf-8') as f:
    classes = [line.strip() for line in f.readlines() if line.strip()]
print(f"检测到 {len(classes)} 个类别: {classes}")

检测到 4 个类别: ['recyclable waste', 'hazardous waste', 'kitchen waste', 'other waste']


In [6]:
"""创建YOLO格式的数据集配置文件"""
dataset_config = {
    'path': str(data_dir.absolute()),
    'train': 'images/train',
    'val': 'images/val', 
    'test': 'images/test',
    'nc': len(classes),
    'names': classes
}

yaml_path = data_dir / "dataset.yaml"
with open(yaml_path, 'w', encoding='utf-8') as f:
    yaml.dump(dataset_config, f, default_flow_style=False, allow_unicode=True)

print(f"数据集配置文件已创建: {yaml_path}")

数据集配置文件已创建: data\dataset.yaml


## 训练模型

In [12]:
def load_pretrained_model(model_size='n'):
    if model_size == 'n':
        model_path = 'models/yolov8n.pt'
    
    model = YOLO(model_path)
    print(f"模型加载完成: {model_path}")
    return model

In [13]:
def train_model(dataset_yaml, epochs=100, batch_size=16, img_size=640, model_size='n'):
    """训练YOLO模型"""
    print("开始训练YOLO模型...")
    
    # 加载预训练模型
    model = load_pretrained_model(model_size)
    
    # 训练参数
    train_args = {
        'data': str(dataset_yaml),
        'epochs': epochs,
        'batch': batch_size,
        'imgsz': img_size,
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'project': 'runs/detect',
        'name': 'waste_detection',
        'save_period': 10,  # 每10个epoch保存一次
        'val': True,  # 每个epoch都进行验证
        'plots': True,  # 生成训练图表
        'verbose': True
    }
    
    print(f"训练设备: {train_args['device']}")
    print(f"训练参数: epochs={epochs}, batch_size={batch_size}, img_size={img_size}")
    
    # 开始训练
    results = model.train(**train_args)
    
    print("训练完成!")
    return model, results

In [14]:
model, train_results = train_model(
    yaml_path,
    epochs=50,  # 可以根据需要调整
    batch_size=16,
    img_size=640,
    model_size='n'  # 可选: n, s, m, l, x (从小到大)
)

开始训练YOLO模型...
模型加载完成: models/yolov8n.pt
训练设备: cuda
训练参数: epochs=50, batch_size=16, img_size=640
Ultralytics 8.3.167  Python-3.11.13 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data\dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=models/yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=waste_detection, nbs=64, nms=False, opset=None, optimize=

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 755k/755k [00:15<00:00, 48.7kB/s]

Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 129 layers, 3,011,628 parameters, 3,011,612 gradients

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5.35M/5.35M [02:09<00:00, 43.4kB/s]


...

      50/50      2.64G     0.7899     0.4915      1.206         19        640: 100%|██████████| 120/120 [00:15<00:00,  7.83it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:03<00:00,  5.41it/s]

                   all        548        757       0.86      0.793      0.873      0.608



50 epochs completed in 0.271 hours.
Optimizer stripped from runs\detect\waste_detection\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\waste_detection\weights\best.pt, 6.2MB

Validating runs\detect\waste_detection\weights\best.pt...
Ultralytics 8.3.167  Python-3.11.13 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:04<00:00,  4.02it/s]


                   all        548        757      0.897      0.773      0.874      0.612
      recyclable waste        209        314      0.892      0.763      0.857      0.622
       hazardous waste        121        152      0.925      0.849      0.914      0.714
         kitchen waste        117        146      0.883      0.685      0.849       0.51
           other waste        109        145      0.886      0.793      0.875        0.6
Speed: 0.2ms preprocess, 2.7ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\detect\waste_detection
训练完成!


## 评估

In [17]:
def evaluate_model(model, yaml_path, split='test'):
    """评估模型性能"""
    print(f"在{split}集上评估模型...")
    # 进行评估
    results = model.val(data=str(yaml_path), split=split)
    
    return results

In [19]:
# 在验证集上评估
print("验证集评估结果:")
val_results = model.val()
print(f"验证集mAP50: {val_results.box.map50:.4f}")
print(f"验证集mAP50-95: {val_results.box.map:.4f}")

验证集评估结果:
Ultralytics 8.3.167  Python-3.11.13 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)
val: Fast image access  (ping: 0.10.1 ms, read: 217.5152.8 MB/s, size: 23.8 KB)


val: Scanning E:\workspace\python\JunkDetectionYOLO\data\labels\val.cache... 548 images, 0 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:04<00:00,  8.14it/s]


                   all        548        757      0.897      0.775      0.873      0.611
      recyclable waste        209        314      0.889      0.762      0.853      0.616
       hazardous waste        121        152      0.922      0.853      0.915      0.714
         kitchen waste        117        146      0.893      0.686      0.848      0.509
           other waste        109        145      0.886        0.8      0.876      0.605
Speed: 0.3ms preprocess, 3.0ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs\detect\waste_detection4
验证集mAP50: 0.8730
验证集mAP50-95: 0.6109


## 测试

In [20]:
results = model("data/images/test/fimg_23.jpg")  # Predict on an image
results[0].show()  # Display results


image 1/1 E:\workspace\python\JunkDetectionYOLO\data\images\test\fimg_23.jpg: 608x640 1 other waste, 34.7ms
Speed: 4.8ms preprocess, 34.7ms inference, 2.6ms postprocess per image at shape (1, 3, 608, 640)


In [22]:
results2 = model("data/images/test/fimg_3652.jpg")
results2[0].show()


image 1/1 E:\workspace\python\JunkDetectionYOLO\data\images\test\fimg_3652.jpg: 480x640 3 recyclable wastes, 99.1ms
Speed: 3.1ms preprocess, 99.1ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)


In [28]:
?results[0]

Type:           Results
String form:   
ultralytics.engine.results.Results object with attributes:
           
           boxes: ultralytics.engine.results.Box <...> reprocess': 4.848800017498434, 'inference': 34.69219998805784, 'postprocess': 2.5514999870210886}
Length:         1
File:           e:\anaconda3\envs\yolo-v8\lib\site-packages\ultralytics\engine\results.py
Docstring:     
A class for storing and manipulating inference results.

This class provides comprehensive functionality for handling inference results from various
Ultralytics models, including detection, segmentation, classification, and pose estimation.
It supports visualization, data export, and various coordinate transformations.

Attributes:
    orig_img (np.ndarray): The original image as a numpy array.
    orig_shape (Tuple[int, int]): Original image shape in (height, width) format.
    boxes (Boxes | None): Detected bounding boxes.
    masks (Masks | None): Segmentation masks.
    probs (Probs | None): Classific

In [31]:
results[0].boxes.data

tensor([[ 14.0896,  59.5534, 316.7597, 255.9985,   0.7869,   3.0000]], device='cuda:0')